<a href="https://colab.research.google.com/github/Inteli-College/2024-2A-T01-CC11-G05/blob/word2vec-pre-trained-model-sprint2/Jonas_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar bibliotecas
import os
from google.colab import drive
from sklearn.preprocessing import LabelEncoder

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import TextVectorization, Embedding

import numpy as np
import pandas as pd

# Montar o Google Drive
drive.mount('/content/drive')

# Carregar os dados
file_path = "/content/drive/Shareddrives/Grupo_05/dataset-m11-tratado.csv"
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataset

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,No,Intencao,Pergunta,Resposta\n,Palavras_japonesas_pergunta,Palavras_japonesas_resposta,Contagem_app_pergunta,Contagem_app_resposta,Abreviacoes
0,1,Como depositar,"['bom', 'dia', 'tudo', 'bem', 'gostar', 'saber...","['bom', 'dia', 'sim', 'senhor', 'poder', 'util...",[],[],0,0,"['eh', 'sr']"
1,2,Como fazer remessa,"['enviar', 'dinheiro', 'japo']","['inscrever', 'servio', 'remessar', 'favor', '...",[],[],0,2,"['app', '24h']"
2,3,Tempo de remessa,"['quanto', 'tempo', 'levar', 'beneficirio', 'r...","['via', 'regro', 'remessa', 'sero', 'pago', 'v...",[],[],0,0,[]
3,4,"Pedido de envio via metodo ""ByPhone""","['bom', 'tarde', 'acabar', 'fazer', 'transfern...","['ir', 'processar', 'solicitacao', 'obrigado',...",[],[],0,0,[]
4,5,"Pedido de envio via metodo ""ByPhone""","['poder', 'fazer', 'remesso', 'iene', 'bbb', '...","['ir', 'processar', 'solicitacao', 'obrigado',...",[],[],0,0,"['bbb', 'rrr']"


In [ ]:
if not os.path.exists("glove.6B.100d.txt"):
  !wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
  !unzip -q glove.6B.zip

In [ ]:
# Organiza aleatoriamente os dados para evitar viés.

data = data.sample(frac = 1)
data.head()

,No,Intencao,Pergunta,Resposta\n,Palavras_japonesas_pergunta,Palavras_japonesas_resposta,Contagem_app_pergunta,Contagem_app_resposta,Abreviacoes
277,278,"Pedido de envio via metodo ""ByPhone""","['ol', 'bom', 'tarde', 'fiz', 'depsizer', 'hoj...","['bom', 'tarde', 'ir', 'processar', 'obrigar']",[],[],0,0,"['yen', 'hh', 'ww']"
155,156,Solicitacao de cartao de remessas,"['carto', 'remessar', 'familha', 'mart']","['bom', 'tarde', 'aps', 'inscrio', 'aprovar', ...",[],[],0,1,['app']
262,263,"Pedido de envio via metodo ""ByPhone""","['fiz', 'depsizer', 'mm', 'll', 'poder', 'veri...","['bom', 'tarde', 'ir', 'processar', 'remesso',...",[],[],0,0,"['fiz', 'mm', 'll']"
365,366,Termos e condicoes do servico,"['ol', 'bom', 'dia', 'dinheiro', 'ainda', 'che...","['ol', 'bom', 'dia', 'dinheiro', 'ainda', 'che...",[],[],0,0,"['lá', 'né', 'lá', 'né']"
189,190,Confirmacao de cambio/taxas,"['mil', 'quanto', 'para', 'enviar', 'hoje', 'B...","['bom', 'tarde', 'real', 'incluir', 'taxa', 'r...",[],[],0,0,"['mil', 'pra']"


In [ ]:
quests = data['Pergunta'].values
labels = data['Intencao'].values

train_data = quests[:int(len(quests) * 0.8)]
train_labels = labels[:int(len(quests) * 0.8)]
test_data = quests[int(len(quests) * 0.8):]
test_labels = labels[int(len(quests) * 0.8):]

print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)
print("\n")
print("Pré-Split:\n", quests[403], "\n", quests[404], "\n")
print("Primeiro treino:", train_data[0])
print("Último treino:", train_data[-1], "\n")
print("Primeiro teste:", test_data[0])
print("Último teste:", test_data[-1])

(404,) (404,) (101,) (101,)


Pré-Split:
 ['enviei', 'dinheiro', 'antes', 'ontem', 'at', 'agora', 'cair', 'conta', 'Brasil'] 
 ['ligar', 'numero', 'nenhum', 'atender', 'nmero', 'funcionar', 'preciso', 'fazer', 'cadastro', 'pessoa', 'Brasil', 'para', 'poder', 'enviar', 'dinheiro', 'fao'] 

Primeiro treino: ['ol', 'bom', 'tarde', 'fiz', 'depsizer', 'hoje', 'valor', 'Yen', 'transferir', 'favor', 'hh', 'ww']
Último treino: ['enviei', 'dinheiro', 'antes', 'ontem', 'at', 'agora', 'cair', 'conta', 'Brasil'] 

Primeiro teste: ['ligar', 'numero', 'nenhum', 'atender', 'nmero', 'funcionar', 'preciso', 'fazer', 'cadastro', 'pessoa', 'Brasil', 'para', 'poder', 'enviar', 'dinheiro', 'fao']
Último teste: ['ol', 'bom', 'dia', 'preciso', 'fazer', 'depsizer', 'chegue', 'Brasil', 'valor', 'real', 'valor', 'preciso', 'depositar', 'j', 'taxa', 'incluso']


In [ ]:
vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=100)
vectorizer.adapt(train_data)

vocabulary  = vectorizer.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

In [ ]:
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
num_tokens = len(vocabulary) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 330 words (356 misses)


In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    trainable=False,
)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

In [ ]:
int_sequences_input = keras.Input(shape=(None,), dtype="int32")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(18, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, None, 100)           │          68,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, None, 128)           │          64,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_5               │ (None, 128)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 18)                  │           2,322 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 151,762 (592.82 KB)

 Trainable params: 82,962 (324.07 KB)

 Non-trainable params: 68,800 (268.75 KB)

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_data])).numpy()
x_val = vectorizer(np.array([[s] for s in test_data])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [ ]:
# Convert string labels to numerical representations

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 297ms/step - acc: 0.1374 - loss: 2.8431 - val_acc: 0.2772 - val_loss: 2.4744
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - acc: 0.3820 - loss: 2.3154 - val_acc: 0.3267 - val_loss: 2.3581
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - acc: 0.4099 - loss: 2.1167 - val_acc: 0.3366 - val_loss: 2.2471
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - acc: 0.4753 - loss: 2.0184 - val_acc: 0.3366 - val_loss: 2.1807
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - acc: 0.5089 - loss: 1.8374 - val_acc: 0.4158 - val_loss: 2.1083
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - acc: 0.5555 - loss: 1.7261 - val_acc: 0.3564 - val_loss: 2.0965
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - acc: 0.5783 - loss: 1.5411 - val_acc: 0.4158 - val_loss: 2.0368
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - acc: 0.6323 - loss: 1.4845 - val_acc: 0.4158 - val_loss: 1.9613
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - acc: 0.6102 - loss: 